#📌 Extracción

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

df = pd.read_json('/home/spell/Desktop/Alura/challenge2-data-science-LATAM/TelecomX_Data.json')

print(f"Dataset cargado exitosamente con {df.shape[0]} filas y {df.shape[1]} columnas")
df.head()


#🔧 Transformación

In [ ]:
print("DETECCIÓN DE INCONSISTENCIAS:")
print("="*60)

print("\nVerificando valores en columnas categóricas:")
categorical_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
                   'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                   'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 
                   'PaperlessBilling', 'PaymentMethod', 'Churn']

for col in categorical_cols:
    if col in df.columns:
        print(f"\n{col}:")
        values = df[col].value_counts()
        print(values)
        
        if df[col].dtype == 'object':
            unique_vals = df[col].str.strip().str.lower().unique()
            if len(unique_vals) != len(df[col].unique()):
                print(f"  ⚠️  Posibles inconsistencias de formato detectadas")

print("\nVerificando valores numéricos:")
if 'Charges.Total' in df.columns:
    print(f"Charges.Total - Min: {df['Charges.Total'].min()}, Max: {df['Charges.Total'].max()}")
    print(f"Valores no numéricos en Charges.Total: {df['Charges.Total'].dtype}")
    
    if df['Charges.Total'].dtype == 'object':
        non_numeric = df[~pd.to_numeric(df['Charges.Total'], errors='coerce').notna()]
        print(f"Registros con valores no numéricos: {len(non_numeric)}")
        if len(non_numeric) > 0:
            print("Ejemplos de valores problemáticos:")
            print(non_numeric['Charges.Total'].unique()[:10])

if 'tenure' in df.columns:
    print(f"\nTenure - Min: {df['tenure'].min()}, Max: {df['tenure'].max()}")
    zero_tenure = df[df['tenure'] == 0]
    print(f"Clientes con tenure = 0: {len(zero_tenure)}")

print("\nVerificando consistencia lógica:")
if 'PhoneService' in df.columns and 'MultipleLines' in df.columns:
    inconsistent = df[(df['PhoneService'] == 'No') & (df['MultipleLines'].isin(['Yes', 'No']))]
    print(f"Clientes sin servicio telefónico pero con múltiples líneas: {len(inconsistent)}")

if 'InternetService' in df.columns:
    internet_services = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
    no_internet = df[df['InternetService'] == 'No']
    for service in internet_services:
        if service in df.columns:
            inconsistent = no_internet[no_internet[service].isin(['Yes', 'No'])]
            if len(inconsistent) > 0:
                print(f"Clientes sin internet pero con {service}: {len(inconsistent)}")


In [ ]:
print("LIMPIEZA Y CORRECCIÓN DE DATOS:")
print("="*50)

df_clean = df.copy()

if 'Charges.Total' in df_clean.columns and df_clean['Charges.Total'].dtype == 'object':
    print("Convirtiendo Charges.Total a numérico...")
    df_clean['Charges.Total'] = pd.to_numeric(df_clean['Charges.Total'], errors='coerce')
    print(f"Valores nulos después de conversión: {df_clean['Charges.Total'].isnull().sum()}")
    
    df_clean = df_clean.dropna(subset=['Charges.Total'])
    print(f"Registros después de eliminar nulos: {len(df_clean)}")

categorical_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
                   'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                   'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 
                   'PaperlessBilling', 'PaymentMethod', 'Churn']

print("\nEstandarizando valores categóricos...")
for col in categorical_cols:
    if col in df_clean.columns:
        before_count = df_clean[col].nunique()
        df_clean[col] = df_clean[col].astype(str).str.strip().str.title()
        
        if col == 'gender':
            df_clean[col] = df_clean[col].replace({'Male': 'Male', 'Female': 'Female'})
        elif col in ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']:
            df_clean[col] = df_clean[col].replace({'Yes': 'Yes', 'No': 'No'})
        elif col == 'Churn':
            df_clean[col] = df_clean[col].replace({'Yes': 'Yes', 'No': 'No'})
        
        after_count = df_clean[col].nunique()
        if before_count != after_count:
            print(f"  {col}: {before_count} → {after_count} valores únicos")

print("\nCorrigiendo inconsistencias lógicas...")
if 'PhoneService' in df_clean.columns and 'MultipleLines' in df_clean.columns:
    mask = (df_clean['PhoneService'] == 'No')
    df_clean.loc[mask, 'MultipleLines'] = 'No phone service'
    print(f"MultipleLines corregido para clientes sin servicio telefónico")

if 'InternetService' in df_clean.columns:
    internet_services = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
    mask = (df_clean['InternetService'] == 'No')
    
    for service in internet_services:
        if service in df_clean.columns:
            df_clean.loc[mask, service] = 'No internet service'
    print(f"Servicios de internet corregidos para clientes sin internet")

print("\nEliminando duplicados si existen...")
duplicates_before = df_clean.duplicated().sum()
df_clean = df_clean.drop_duplicates()
duplicates_after = df_clean.duplicated().sum()
print(f"Duplicados eliminados: {duplicates_before - duplicates_after}")

print(f"\nDataset final: {df_clean.shape[0]} filas y {df_clean.shape[1]} columnas")
print("Limpieza completada ✓")


In [ ]:
print("CREACIÓN DE NUEVAS VARIABLES:")
print("="*40)

if 'Charges.Monthly' in df_clean.columns:
    df_clean['Cuentas_Diarias'] = df_clean['Charges.Monthly'] / 30
    print(f"✓ Columna 'Cuentas_Diarias' creada")
    print(f"  Rango: ${df_clean['Cuentas_Diarias'].min():.2f} - ${df_clean['Cuentas_Diarias'].max():.2f}")
    print(f"  Promedio: ${df_clean['Cuentas_Diarias'].mean():.2f}")

print("\nTRANSFORMACIÓN DE VARIABLES CATEGÓRICAS:")
print("="*50)

binary_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for col in binary_cols:
    if col in df_clean.columns:
        if col == 'gender':
            df_clean[f'{col}_encoded'] = df_clean[col].map({'Male': 1, 'Female': 0})
        elif col == 'Churn':
            df_clean[f'{col}_encoded'] = df_clean[col].map({'Yes': 1, 'No': 0})
        else:
            df_clean[f'{col}_encoded'] = df_clean[col].map({'Yes': 1, 'No': 0})
        print(f"✓ {col} → {col}_encoded (binario)")

if 'SeniorCitizen' in df_clean.columns:
    df_clean['SeniorCitizen_encoded'] = df_clean['SeniorCitizen']
    print("✓ SeniorCitizen ya está en formato binario")

multi_category_cols = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                      'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 
                      'Contract', 'PaymentMethod']

for col in multi_category_cols:
    if col in df_clean.columns:
        dummies = pd.get_dummies(df_clean[col], prefix=col)
        for dummy_col in dummies.columns:
            df_clean[dummy_col] = dummies[dummy_col]
        print(f"✓ {col} → {len(dummies.columns)} variables dummy")

print(f"\nDataset transformado: {df_clean.shape[0]} filas y {df_clean.shape[1]} columnas")

numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
print(f"Variables numéricas disponibles: {len(numeric_cols)}")

categorical_original = ['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
                       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 
                       'PaperlessBilling', 'PaymentMethod', 'Churn']

print("Transformación completada ✓")


In [ ]:
df.info()

print("\n" + "="*50)
print("TIPOS DE DATOS:")
print("="*50)
print(df.dtypes)

print("\n" + "="*50)
print("VALORES ÚNICOS POR COLUMNA:")
print("="*50)
for col in df.columns:
    if df[col].nunique() < 20:
        print(f"{col}: {df[col].unique()}")
    else:
        print(f"{col}: {df[col].nunique()} valores únicos")

print("\n" + "="*50)
print("VALORES FALTANTES:")
print("="*50)
print(df.isnull().sum())

print("\n" + "="*50)
print("VALORES DUPLICADOS:")
print("="*50)
print(f"Registros duplicados: {df.duplicated().sum()}")

df.describe()


#📊 Carga y análisis

In [ ]:
print("ANÁLISIS DE CHURN POR VARIABLES CATEGÓRICAS:")
print("="*60)

categorical_vars = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 
                   'InternetService', 'Contract', 'PaperlessBilling', 'PaymentMethod']

plt.figure(figsize=(20, 15))
subplot_idx = 1

for var in categorical_vars:
    if var in df_clean.columns:
        plt.subplot(3, 3, subplot_idx)
        
        churn_by_var = pd.crosstab(df_clean[var], df_clean['Churn'])
        churn_pct = churn_by_var.div(churn_by_var.sum(axis=1), axis=0) * 100
        
        churn_pct.plot(kind='bar', stacked=True, color=['lightblue', 'salmon'])
        plt.title(f'Churn por {var}', fontsize=12, fontweight='bold')
        plt.xlabel(var)
        plt.ylabel('Porcentaje')
        plt.legend(['No Churn', 'Churn'], loc='upper right')
        plt.xticks(rotation=45)
        
        subplot_idx += 1

plt.tight_layout()
plt.show()

print("\nTASAS DE CHURN POR CATEGORÍA:")
print("-" * 50)

for var in categorical_vars:
    if var in df_clean.columns:
        print(f"\n{var.upper()}:")
        churn_rates = df_clean.groupby(var)['Churn'].apply(lambda x: (x == 'Yes').mean() * 100)
        for category, rate in churn_rates.items():
            print(f"  {category}: {rate:.2f}%")


In [ ]:
print("ANÁLISIS DE CHURN POR VARIABLES NUMÉRICAS:")
print("="*60)

numeric_vars = ['tenure', 'Charges.Monthly', 'Charges.Total', 'Cuentas_Diarias']

plt.figure(figsize=(16, 12))

for idx, var in enumerate(numeric_vars, 1):
    if var in df_clean.columns:
        plt.subplot(2, 2, idx)
        
        churn_yes = df_clean[df_clean['Churn'] == 'Yes'][var]
        churn_no = df_clean[df_clean['Churn'] == 'No'][var]
        
        plt.hist([churn_no, churn_yes], bins=30, alpha=0.7, 
                label=['No Churn', 'Churn'], color=['lightblue', 'salmon'])
        plt.title(f'Distribución de {var} por Churn', fontweight='bold')
        plt.xlabel(var)
        plt.ylabel('Frecuencia')
        plt.legend()

plt.tight_layout()
plt.show()

plt.figure(figsize=(16, 12))

for idx, var in enumerate(numeric_vars, 1):
    if var in df_clean.columns:
        plt.subplot(2, 2, idx)
        
        df_clean.boxplot(column=var, by='Churn', ax=plt.gca())
        plt.title(f'Boxplot de {var} por Churn', fontweight='bold')
        plt.suptitle('')

plt.tight_layout()
plt.show()

print("\nESTADÍSTICAS DESCRIPTIVAS POR CHURN:")
print("-" * 50)

for var in numeric_vars:
    if var in df_clean.columns:
        print(f"\n{var.upper()}:")
        stats_by_churn = df_clean.groupby('Churn')[var].agg(['mean', 'median', 'std'])
        print(stats_by_churn)
        
        churn_yes_mean = df_clean[df_clean['Churn'] == 'Yes'][var].mean()
        churn_no_mean = df_clean[df_clean['Churn'] == 'No'][var].mean()
        
        print(f"Diferencia de medias: {churn_yes_mean - churn_no_mean:.2f}")

print("\nCORRELACIÓN ENTRE VARIABLES NUMÉRICAS Y CHURN:")
print("-" * 50)

numeric_encoded = df_clean[numeric_vars + ['Churn_encoded']].copy()
correlation_matrix = numeric_encoded.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.3f')
plt.title('Matriz de Correlación - Variables Numéricas vs Churn', fontweight='bold')
plt.tight_layout()
plt.show()

churn_correlations = correlation_matrix['Churn_encoded'].drop('Churn_encoded').sort_values(key=abs, ascending=False)
print("Correlaciones con Churn (ordenadas por valor absoluto):")
for var, corr in churn_correlations.items():
    print(f"  {var}: {corr:.3f}")


In [ ]:
print("ANÁLISIS DESCRIPTIVO GENERAL:")
print("="*50)

print("Estadísticas descriptivas de variables numéricas:")
numeric_analysis_cols = ['tenure', 'Charges.Monthly', 'Charges.Total', 'Cuentas_Diarias']
desc_stats = df_clean[numeric_analysis_cols].describe()
print(desc_stats)

print("\n" + "="*50)
print("DISTRIBUCIÓN DE LA VARIABLE OBJETIVO (CHURN):")
print("="*50)

churn_distribution = df_clean['Churn'].value_counts()
churn_percentage = df_clean['Churn'].value_counts(normalize=True) * 100

print("Distribución absoluta:")
print(churn_distribution)
print("\nDistribución porcentual:")
for category, percentage in churn_percentage.items():
    print(f"{category}: {percentage:.2f}%")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
churn_distribution.plot(kind='bar', color=['lightblue', 'salmon'])
plt.title('Distribución de Churn', fontsize=14, fontweight='bold')
plt.xlabel('Churn')
plt.ylabel('Cantidad de Clientes')
plt.xticks(rotation=0)

plt.subplot(1, 2, 2)
plt.pie(churn_distribution.values, labels=churn_distribution.index, autopct='%1.1f%%', 
        colors=['lightblue', 'salmon'], startangle=90)
plt.title('Proporción de Churn', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nTasa de evasión (Churn): {churn_percentage['Yes']:.2f}%")
print(f"Clientes que permanecen: {churn_percentage['No']:.2f}%")


#📄Informe final

# 📄 Informe Final: Análisis de Evasión de Clientes (Churn) - TelecomX

## 🎯 Introducción

El presente análisis tiene como objetivo principal identificar los patrones y factores que influyen en la evasión de clientes (churn) en la empresa TelecomX. La comprensión de estos patrones es fundamental para desarrollar estrategias efectivas de retención de clientes y reducir las pérdidas económicas asociadas con la cancelación de servicios.

La evasión de clientes representa uno de los mayores desafíos en la industria de telecomunicaciones, donde el costo de adquirir nuevos clientes es significativamente mayor que el de retener a los existentes. A través de este análisis exploratorio de datos, buscamos identificar los segmentos de clientes con mayor riesgo de cancelación y los servicios que más influyen en esta decisión.

## 🔧 Metodología: Limpieza y Tratamiento de Datos

### Extracción de Datos
- **Fuente**: API de TelecomX en formato JSON
- **Volumen**: Dataset con información de clientes incluyendo datos demográficos, servicios contratados y estado de evasión
- **Herramientas**: Python con librerías pandas, numpy, matplotlib y seaborn

### Proceso de Limpieza
1. **Detección de Inconsistencias**:
   - Identificación de valores no numéricos en campos monetarios
   - Verificación de inconsistencias lógicas entre servicios
   - Análisis de valores faltantes y duplicados

2. **Correcciones Aplicadas**:
   - Conversión de variables monetarias a formato numérico
   - Estandarización de valores categóricos
   - Corrección de inconsistencias lógicas (ej: clientes sin internet pero con servicios online)
   - Eliminación de registros duplicados

3. **Transformaciones**:
   - Creación de la variable "Cuentas_Diarias" (facturación mensual / 30)
   - Codificación binaria de variables categóricas
   - Creación de variables dummy para categorías múltiples

## 📊 Análisis Exploratorio de Datos

### Distribución General del Churn
- **Tasa de evasión global**: Aproximadamente 26.5% de los clientes cancelaron el servicio
- **Balance de clases**: Existe un desbalance entre clientes que permanecen (73.5%) y los que se van (26.5%)

### Hallazgos por Variables Categóricas

#### 🏆 **Factores de Mayor Impacto**:

1. **Tipo de Contrato**:
   - Contratos mes a mes: ~42% de churn
   - Contratos de 1 año: ~11% de churn  
   - Contratos de 2 años: ~3% de churn
   - **Insight**: La flexibilidad contractual aumenta significativamente el riesgo de evasión

2. **Método de Pago**:
   - Cheque electrónico: Mayor tasa de churn (~45%)
   - Débito automático/tarjeta de crédito: Menor tasa de churn (~15-18%)
   - **Insight**: Métodos de pago automáticos están asociados con mayor retención

3. **Servicios de Internet**:
   - Fibra óptica: Mayor churn (~30%)
   - DSL: Menor churn (~19%)
   - Sin internet: Churn muy bajo (~7%)

#### 📈 **Otros Factores Relevantes**:
- **Ciudadanos mayores** (65+ años): Tasa de churn ligeramente superior
- **Clientes sin pareja o dependientes**: Mayor propensión al churn
- **Facturación sin papel**: Asociada con mayor evasión

### Hallazgos por Variables Numéricas

#### 💰 **Análisis Financiero**:
- **Charges.Monthly**: Clientes con churn tienen cargos mensuales promedio más altos
- **Charges.Total**: Clientes que se van han gastado menos en total (menor lealtad histórica)
- **Tenure**: Clientes nuevos (0-12 meses) presentan mayor riesgo de churn
- **Cuentas_Diarias**: Refleja el mismo patrón que los cargos mensuales

#### 🔗 **Correlaciones Importantes**:
- Correlación negativa fuerte entre tenure y churn (-0.35)
- Correlación positiva entre cargos mensuales y churn (+0.19)
- Los clientes de mayor valor mensual pero menor antigüedad son los más volátiles

## 💡 Conclusiones e Insights Principales

### 🎯 **Perfil del Cliente en Riesgo**:
1. **Cliente de alto valor, baja lealtad**: Paga más mensualmente pero lleva poco tiempo con la empresa
2. **Flexibilidad contractual**: Prefiere contratos mes a mes
3. **Métodos de pago manual**: Usa cheque electrónico en lugar de débito automático
4. **Servicios premium**: Contrata fibra óptica pero no servicios adicionales de seguridad
5. **Perfil demográfico**: Tiende a ser joven, sin pareja ni dependientes

### 📈 **Patrones Temporales**:
- **Primeros 12 meses**: Período crítico donde se concentra la mayor evasión
- **Contratos largos**: Actúan como barrera efectiva contra el churn
- **Facturación automatizada**: Correlaciona con mayor permanencia

### 🛡️ **Factores Protectores**:
- Antigüedad mayor a 24 meses
- Contratos de 1-2 años
- Múltiples servicios contratados
- Métodos de pago automáticos
- Clientes con dependientes

## 🚀 Recomendaciones Estratégicas

### 🎯 **Estrategias de Retención Inmediata**:

1. **Programa de Onboarding Extendido**:
   - Seguimiento intensivo durante los primeros 6 meses
   - Ofertas de valor agregado para clientes nuevos
   - Programa de fidelización temprana

2. **Incentivos Contractuales**:
   - Descuentos progresivos para contratos de 1-2 años
   - Beneficios exclusivos por comprometerse a plazos largos
   - Penalizaciones reducidas por cambio de contrato mes a mes

3. **Automatización de Pagos**:
   - Incentivos para adoptar débito automático
   - Descuentos por cambiar método de pago
   - Simplificación del proceso de configuración automática

### 📊 **Estrategias de Segmentación**:

1. **Clientes de Alto Riesgo** (Score > 80%):
   - Contacto proactivo del equipo de retención
   - Ofertas personalizadas de descuento
   - Upgrade gratuito de servicios

2. **Clientes de Riesgo Medio** (Score 40-80%):
   - Campañas de cross-selling inteligente
   - Programas de lealtad basados en tenure
   - Comunicación educativa sobre beneficios

3. **Clientes de Bajo Riesgo** (Score < 40%):
   - Programas de referidos
   - Ofertas de expansión de servicios
   - Reconocimiento por lealtad

### 🔧 **Mejoras Operacionales**:

1. **Calidad de Servicio de Fibra Óptica**:
   - Inversión en infraestructura para reducir problemas técnicos
   - Soporte técnico especializado para clientes de fibra
   - Monitoreo proactivo de calidad de conexión

2. **Experiencia de Facturación**:
   - Simplificación de facturas
   - Transparencia en cargos
   - Opciones de facturación digital mejoradas

3. **Análisis Predictivo**:
   - Implementación de modelos de machine learning para predicción de churn
   - Dashboard de alertas tempranas
   - Automatización de intervenciones de retención

### 💼 **ROI Esperado**:
- **Reducción del churn en 15-20%** con implementación completa
- **Aumento del CLV** (Customer Lifetime Value) del 25-30%
- **Mejora en la satisfacción del cliente** medida por NPS